# Forward Chaining Inference Engine

Source: https://en.wikipedia.org/wiki/Forward_chaining

Forward chaining (or forward reasoning) is one of the two main methods of reasoning when using an inference engine and can be described logically as repeated application of modus ponens. Forward chaining is a popular implementation strategy for expert systems, business, and production rule systems. 
Forward chaining starts with the available data and uses inference rules to extract more data (from an end user, for example) until a goal is reached. An inference engine using forward chaining searches the inference rules until it finds one where the antecedent (If clause) is known to be true. When such a rule is found, the engine can conclude, or infer, the consequent (Then clause), resulting in the addition of new information to its data.
The Inference engine will iterate through this process until a goal is reached.
Suppose that the goal is to conclude the colour of a pet named Fritz, given that he croaks and eats flies, and that the rule base contains the following four rules:
- If X croaks and X eats flies - Then X is a frog
- If X chirps and X sings - Then X is a canary
- If X is a frog - Then X is green
- If X is a canary - Then X is yellow

With forward reasoning, the inference engine can derive that Fritz is green in a series of steps.
The name "forward chaining" comes from the fact that the inference engine starts with the data and reasons its way to the answer. Because the data determines which rules are selected and used, this method is called data-driven. The forward chaining approach is often employed by expert system. 
One of the advantages of forward-chaining over backward-chaining is that the reception of new data can trigger new inferences, which makes the engine better suited to dynamic situations in which conditions are likely to change.

The above text is from https://en.wikipedia.org/wiki/Forward_chaining

# A Forward Chaining Rule Engine - Demo

In [1]:
import pandas as pd
from asteval import Interpreter
aeval = Interpreter()

In [2]:
def assert_fact(ruleSet= None, entity=None, subject=None, antecedent=None, debug=False, debugL2=False):    
    """
    external procs should not call assert_fact() directly. they should use evaluate_fact()
    ONLY consequent functions can call assert_fact()
    
    entity: e.g. user_id, user_name
    ruleset: which rules to use for the evaluation (in a pandas dataframe)
    subject: which rule/s with that subject to use for evaluation
    antecedent: what is the value to compare with vs the values that exists in the ruleset.
    
    """
    ruleset_hexStr = "72756c65736574"  # hex string for 'ruleset'
    ruleset = bytearray.fromhex(ruleset_hexStr).decode()  # to avoid dataframe from auto display when imputing as fn arg
    vars = [ruleset, entity, subject,antecedent,debug,debugL2]  # standard set of vars when imputing as args into consequent function
    encode_var = lambda x: '"'+x+'"' if isinstance(x,str) else str(x)
    arg_end = ')'
    c_list = []
    
#     if debug: 
#         print("All rules in this rule set:"), display(ruleSet)    
    
    #oc = ruleSet.operator==operator
    #ac = ruleSet.antecedent==antecedent    
    try:
        # find the rule/s to use if it exists in the rule set
        # the rule/s should be those that match the subject 
        if debug: print('\nAssert Fact:\n\t', entity, subject, antecedent,'\n')
        
        sc = ruleSet.subject==subject
        rules = ruleSet.loc[sc]        
        if debug: print('rule to be used:\n',rules)        
        
        # test the fact against the rule
        # the eval() method - substitute the value of var into the string at eval() time    
        # the aeval() method - substitue the value of var into the string before call aeval()
        # e.g. age = 17
        #      eval('age < 20')  # 17 is substitute when eval() is executed
        #      aeval('17 < 20')  # 17 must be in the string before calling aeval()


        for index, row in rules.iterrows():                    
            if ('and' not in row.operator.lower()) and ('or' not in row.operator.lower()):
                if debugL2: print('this rule has no "and" and no "or".')
                # straight forward, no 'and'
                r = [str(x) for x in row[:-1]] 
                if debug: print(r)
                r[0] = str(antecedent) 
                r = ' '.join(r)
            elif ('and' in row.operator.lower()) or ('or' in row.operator.lower()) :
                if debugL2: print('this rule has "and" or "or".')                 
                t = [str(x) for x in row[:-1]]
                if debug: print(t)
                o=t[1].split()
                a=t[2].split(',')
                limit = o.count('and')
                limit+= o.count('or')
                limit +=1
                r = ''
                for i in range(limit):    
                    r += str(antecedent).strip() + str(o[i*2]).strip() + str(a[i].strip())
                    if i<limit-1:
                        r += ' ' + o[2*i+1] + ' '
            
            if debugL2: print('\ncheck if:', r)               
            
            result = aeval(r)   # result = eval(r)                                    
            if result:                
                # execute the consequent/s if the fact passes the rule testing and EXIT                                 
                if debug: print('rule passed, consequent is triggered.')
                c = row.consequent
                num_of_fn = c.split(',')                
                for fn in num_of_fn:                    
                    c = fn.strip()                    
                    if debugL2: print('before imputing arguments, consequent is: ',c)
                    c = c.split(')')[0]
                    for var in vars: 
                        c += encode_var(var) +','
                    c=c[:-1]        
                    c+=arg_end
                    c = c.replace('"', '', 2) 
                    next_step = eval(c) 
                    if next_step is not None:
                        c_list.append(next_step)
                    if debugL2: print('after imputing arguments, c is:',c)                                                        
                #return c_list
            else:
                # consequent not triggreed, EXIT
                if debug: print('rule not passed, no consequent.\n')
                #return c_list                        
            del r                        
    except Exception as e:
        # not found
        if debug: print('Rule not found or',e)
            
    return c_list


def evaluate_fact(ruleset,entity,subject,antecedent, debug=False, debugL2=False):
    cont = True
    results = assert_fact(ruleset,entity,subject,antecedent, debug=debug, debugL2=debugL2)    
    while cont:
        #if results is not None:            
        if len(results)>0:
            if debug: 
                print()
                print(results)    
            for r in results:
                results=eval(r)
        else: cont = False

In [3]:
# Consequent Functions for Kermit the Frog
#def user_is_underAged(ruleset,entity,subject,antecedent, debug=False, debugL2=False):
    
def is_a_frog(ruleset,entity,subject,antecedent, debug=False, debugL2=False):
    print('{} is a frog.'.format(entity))        
    #return "assert_fact(ruleset,entity,subject='frog',antecedent=True,debug=False,debugL2=True)"
    return "assert_fact(ruleset,entity,subject='frog',antecedent=True)"
       
def is_a_canary(ruleset,entity,subject,antecedent, debug=False, debugL2=False):
    print('{} is a canary.\n'.format(entity))

def is_green(ruleset,entity,subject,antecedent, debug=False, debugL2=False):
    print('{} is green.'.format(entity))
    #return "assert_fact(ruleset,entity,subject='swim',antecedent=True)"
    return None
    
    
def can_swim(ruleset,entity,subject,antecedent, debug=False, debugL2=False):
    print('{} can swim.'.format(entity))
    #return "assert_fact(ruleset,entity,subject='comfortable in water',antecedent=True,debug=False,debugL2=True)"    
    return "assert_fact(ruleset,entity,subject='comfortable in water',antecedent=True)"    

def comfortable_in_water(ruleset,entity,subject,antecedent, debug=False, debugL2=False):
    print('{} is comfortable in water.'.format(entity))
    return None
    
def is_yellow(ruleset,entity,subject,antecedent, debug=False, debugL2=False):
    print('{} is yellow.'.format(entity))

# Rules for Kermit the Frog   
data = {
    'subject': ['croaks and eats flies','chirps and sings','frog','canary','swim','comfortable in water'],
    'operator':['==','==','==','==','==','=='],
    'antecedent':['True','True','True','True','True','True'],
    'consequent':['is_a_frog()','is_a_canary()','is_green(), can_swim()','is_yellow()','can_swim()','comfortable_in_water()']
}
df = pd.DataFrame(data)
display(df)

,subject,operator,antecedent,consequent
0,croaks and eats flies,==,True,is_a_frog()
1,chirps and sings,==,True,is_a_canary()
2,frog,==,True,"is_green(), can_swim()"
3,canary,==,True,is_yellow()
4,swim,==,True,can_swim()
5,comfortable in water,==,True,comfortable_in_water()


### Demo - Kermit is a frog

In [4]:
ruleset = df
entity = 'Kermit'
subject = 'croaks and eats flies'
antecedent = True
evaluate_fact(ruleset,entity,subject,antecedent, debug=False, debugL2=False)            

Kermit is a frog.
Kermit is green.
Kermit can swim.
Kermit is comfortable in water.


In [5]:
ruleset = df
entity = 'Kermit'
subject = 'croaks and eats flies'
antecedent = True
evaluate_fact(ruleset,entity,subject,antecedent, debug=True, debugL2=False)            


Assert Fact:
	 Kermit croaks and eats flies True 

rule to be used:
                  subject operator antecedent   consequent
0  croaks and eats flies       ==       True  is_a_frog()
['croaks and eats flies', '==', 'True']
rule passed, consequent is triggered.
Kermit is a frog.

["assert_fact(ruleset,entity,subject='frog',antecedent=True)"]
Kermit is green.
Kermit can swim.

["assert_fact(ruleset,entity,subject='comfortable in water',antecedent=True)"]
Kermit is comfortable in water.
